In [2]:
import xarray as xr
import numpy as np
from scipy import interpolate
from sklearn.metrics import mean_absolute_error, r2_score

# Load datasets
full_ds_nonull = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(0, 50))

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
full_ds_nonull[variable_name] = full_ds_nonull[variable_name] / 1000000000000000

full_ds_less1 = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/nulls_less1perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name] = full_ds_less1[variable_name] / 1000000000000000

# Create a nan mask
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)
del full_ds_less1, unique_times, index

inpainted_results = np.empty_like(full_ds_nonull[f"{variable_name}_masked"])

for i in range(100):
    no2 = full_ds_nonull.isel(time=i)[f"{variable_name}_masked"]
    no2_array = np.array(no2)
    x = np.arange(0, no2_array.shape[1])
    y = np.arange(0, no2_array.shape[0])
    #mask invalid values
    no2_array = np.ma.masked_invalid(no2_array)
    xx, yy = np.meshgrid(x, y)
    #get only the valid values
    x1 = xx[~no2_array.mask]
    y1 = yy[~no2_array.mask]
    newarr = no2_array[~no2_array.mask]

    GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy),
                                 method='linear',
                                  fill_value = float(no2.median()))
    
    # Assign the inpainted result to the new variable for each time step
    inpainted_results[i, :, :] = GD1

full_ds_nonull[f"{variable_name}_withinpaint"] = (('time', 'latitude', 'longitude'), inpainted_results)

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

biharmonic_values = full_ds_nonull[f"{variable_name}_withinpaint"].values
inpainted_flat = biharmonic_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

mae:  0.8216167400523553
r2:  0.8548101644135144


In [3]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
from sklearn.metrics import mean_absolute_error, r2_score

# Load datasets
full_ds_nonull = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(0, 50))

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
full_ds_nonull[variable_name] = full_ds_nonull[variable_name] / 1000000000000000

full_ds_less1 = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/nulls_less5perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name] = full_ds_less1[variable_name] / 1000000000000000

# Create a nan mask
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)
del full_ds_less1, unique_times, index

inpainted_results = np.empty_like(full_ds_nonull[f"{variable_name}_masked"])

for i in range(100):
    no2 = full_ds_nonull.isel(time=i)[f"{variable_name}_masked"]
    no2_array = np.array(no2)
    x = np.arange(0, no2_array.shape[1])
    y = np.arange(0, no2_array.shape[0])
    #mask invalid values
    no2_array = np.ma.masked_invalid(no2_array)
    xx, yy = np.meshgrid(x, y)
    #get only the valid values
    x1 = xx[~no2_array.mask]
    y1 = yy[~no2_array.mask]
    newarr = no2_array[~no2_array.mask]

    GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy),
                                 method='linear',
                                  fill_value = float(no2.median()))
    
    # Assign the inpainted result to the new variable for each time step
    inpainted_results[i, :, :] = GD1

full_ds_nonull[f"{variable_name}_withinpaint"] = (('time', 'latitude', 'longitude'), inpainted_results)

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

biharmonic_values = full_ds_nonull[f"{variable_name}_withinpaint"].values
inpainted_flat = biharmonic_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

mae:  0.9059556546053679
r2:  0.7362791858663251


In [4]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
from sklearn.metrics import mean_absolute_error, r2_score

# Load datasets
full_ds_nonull = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(0, 50))

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
full_ds_nonull[variable_name] = full_ds_nonull[variable_name] / 1000000000000000

full_ds_less1 = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/nulls_less10perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name] = full_ds_less1[variable_name] / 1000000000000000

# Create a nan mask
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)
del full_ds_less1, unique_times, index

inpainted_results = np.empty_like(full_ds_nonull[f"{variable_name}_masked"])

for i in range(100):
    no2 = full_ds_nonull.isel(time=i)[f"{variable_name}_masked"]
    no2_array = np.array(no2)
    x = np.arange(0, no2_array.shape[1])
    y = np.arange(0, no2_array.shape[0])
    #mask invalid values
    no2_array = np.ma.masked_invalid(no2_array)
    xx, yy = np.meshgrid(x, y)
    #get only the valid values
    x1 = xx[~no2_array.mask]
    y1 = yy[~no2_array.mask]
    newarr = no2_array[~no2_array.mask]

    GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy),
                                 method='linear',
                                  fill_value = float(no2.median()))
    
    # Assign the inpainted result to the new variable for each time step
    inpainted_results[i, :, :] = GD1

full_ds_nonull[f"{variable_name}_withinpaint"] = (('time', 'latitude', 'longitude'), inpainted_results)

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

biharmonic_values = full_ds_nonull[f"{variable_name}_withinpaint"].values
inpainted_flat = biharmonic_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

mae:  0.9880228446802127
r2:  0.5280566693220173


In [5]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
from sklearn.metrics import mean_absolute_error, r2_score

# Load datasets
full_ds_nonull = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(0, 50))

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
full_ds_nonull[variable_name] = full_ds_nonull[variable_name] / 1000000000000000

full_ds_less1 = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/nulls_less25perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name] = full_ds_less1[variable_name] / 1000000000000000

# Create a nan mask
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)
del full_ds_less1, unique_times, index

inpainted_results = np.empty_like(full_ds_nonull[f"{variable_name}_masked"])

for i in range(100):
    no2 = full_ds_nonull.isel(time=i)[f"{variable_name}_masked"]
    no2_array = np.array(no2)
    x = np.arange(0, no2_array.shape[1])
    y = np.arange(0, no2_array.shape[0])
    #mask invalid values
    no2_array = np.ma.masked_invalid(no2_array)
    xx, yy = np.meshgrid(x, y)
    #get only the valid values
    x1 = xx[~no2_array.mask]
    y1 = yy[~no2_array.mask]
    newarr = no2_array[~no2_array.mask]

    GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy),
                                 method='linear',
                                  fill_value = float(no2.median()))
    
    # Assign the inpainted result to the new variable for each time step
    inpainted_results[i, :, :] = GD1

full_ds_nonull[f"{variable_name}_withinpaint"] = (('time', 'latitude', 'longitude'), inpainted_results)

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

biharmonic_values = full_ds_nonull[f"{variable_name}_withinpaint"].values
inpainted_flat = biharmonic_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

mae:  1.207084981290664
r2:  0.5988266291882316


In [6]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
from sklearn.metrics import mean_absolute_error, r2_score

# Load datasets
full_ds_nonull = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(0, 50))

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
full_ds_nonull[variable_name] = full_ds_nonull[variable_name] / 1000000000000000

full_ds_less1 = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/nulls_less50perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name] = full_ds_less1[variable_name] / 1000000000000000

# Create a nan mask
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)
del full_ds_less1, unique_times, index

inpainted_results = np.empty_like(full_ds_nonull[f"{variable_name}_masked"])

for i in range(100):
    no2 = full_ds_nonull.isel(time=i)[f"{variable_name}_masked"]
    no2_array = np.array(no2)
    x = np.arange(0, no2_array.shape[1])
    y = np.arange(0, no2_array.shape[0])
    #mask invalid values
    no2_array = np.ma.masked_invalid(no2_array)
    xx, yy = np.meshgrid(x, y)
    #get only the valid values
    x1 = xx[~no2_array.mask]
    y1 = yy[~no2_array.mask]
    newarr = no2_array[~no2_array.mask]

    GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy),
                                 method='linear',
                                  fill_value = float(no2.median()))
    
    # Assign the inpainted result to the new variable for each time step
    inpainted_results[i, :, :] = GD1

full_ds_nonull[f"{variable_name}_withinpaint"] = (('time', 'latitude', 'longitude'), inpainted_results)

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

biharmonic_values = full_ds_nonull[f"{variable_name}_withinpaint"].values
inpainted_flat = biharmonic_values[nan_mask.values]

mae = mean_absolute_error(original_flat, inpainted_flat)
r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

mae:  1.2458653836494875
r2:  0.08830768466811256


In [9]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
from sklearn.metrics import mean_absolute_error, r2_score

# Load datasets
full_ds_nonull = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/no_nulls.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_nonull['time'].values, return_index=True)
full_ds_nonull = full_ds_nonull.isel(time=index)
# full_ds_nonull = full_ds_nonull.isel(time=slice(0, 50))

variable_name = 'vertical_column_troposphere'  # Replace with the actual variable name
full_ds_nonull[variable_name] = full_ds_nonull[variable_name] / 1000000000000000

full_ds_less1 = xr.open_dataset(f'../../data/tempo_data/inpaint_experiments/nulls_over50perc.nc', engine='netcdf4')
unique_times, index = np.unique(full_ds_less1['time'].values, return_index=True)
full_ds_less1 = full_ds_less1.isel(time=index)
full_ds_less1[variable_name] = full_ds_less1[variable_name] / 1000000000000000

# Create a nan mask
nan_mask = full_ds_less1[variable_name].isnull()
time_dimension_nonulls = full_ds_nonull['time']
nan_mask = nan_mask.assign_coords(time=time_dimension_nonulls)
full_ds_nonull[f"{variable_name}_masked"] = full_ds_nonull[variable_name].where(~nan_mask)
del full_ds_less1, unique_times, index

inpainted_results = np.empty_like(full_ds_nonull[f"{variable_name}_masked"])

for i in range(100):
    no2 = full_ds_nonull.isel(time=i)[f"{variable_name}_masked"]
    no2_array = np.array(no2)
    x = np.arange(0, no2_array.shape[1])
    y = np.arange(0, no2_array.shape[0])
    #mask invalid values
    no2_array = np.ma.masked_invalid(no2_array)
    xx, yy = np.meshgrid(x, y)
    #get only the valid values
    x1 = xx[~no2_array.mask]
    y1 = yy[~no2_array.mask]
    newarr = no2_array[~no2_array.mask]

    GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy),
                                 method='linear',
                                  fill_value = float(no2.median()))
    
    # Assign the inpainted result to the new variable for each time step
    inpainted_results[i, :, :] = GD1

full_ds_nonull[f"{variable_name}_withinpaint"] = (('time', 'latitude', 'longitude'), inpainted_results)

# # Extract original values (masked by nan_mask)
original_values = full_ds_nonull.where(nan_mask)[variable_name].values
original_flat = original_values[nan_mask.values]

biharmonic_values = full_ds_nonull[f"{variable_name}_withinpaint"].values
inpainted_flat = biharmonic_values[nan_mask.values]
m = ~np.isnan(inpainted_flat)


mae = mean_absolute_error(original_flat[m], inpainted_flat[m])
r2 = r2_score(original_flat[m], inpainted_flat[m])

# mae = mean_absolute_error(original_flat, inpainted_flat)
# r2 = r2_score(original_flat, inpainted_flat)

print("mae: ", mae)
print("r2: ", r2)

ValueError: No points given

In [ ]:
import matplotlib.pyplot as plt
# # Define your latitude and longitude bounds
# lat_min, lat_max = 28.6, 33.4  # Example latitude range
# lon_min, lon_max = -98.9, -88.3  # Example longitude range

# Select one hour of temperature data (e.g., the first timestamp)
hour_index = 0 #Change this to select a different hour if desired
temperature_data = full_ds_nonull[f"{variable_name}_withinpaint"].isel(time=0)
# temperature_data = ds['Percent_Tree_Cover']

# Plot the data with switched axes
plt.figure(figsize=(10, 6))

# Plot with latitude on x-axis and longitude on y-axis
temperature_data.T.plot(
    cmap="coolwarm",  # Colormap for temperature visualization
    cbar_kwargs={'label': 'Temperature (K)'}  # Add color bar label
)

# Update axis labels
plt.xlabel("Latitude")
plt.ylabel("Longitude")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
# # Define your latitude and longitude bounds
# lat_min, lat_max = 28.6, 33.4  # Example latitude range
# lon_min, lon_max = -98.9, -88.3  # Example longitude range

# Select one hour of temperature data (e.g., the first timestamp)
hour_index = 0 #Change this to select a different hour if desired
temperature_data = test1['new']
# temperature_data = ds['Percent_Tree_Cover']

# Plot the data with switched axes
plt.figure(figsize=(10, 6))

# Plot with latitude on x-axis and longitude on y-axis
temperature_data.T.plot(
    cmap="coolwarm",  # Colormap for temperature visualization
    cbar_kwargs={'label': 'Temperature (K)'}  # Add color bar label
)

# Update axis labels
plt.xlabel("Latitude")
plt.ylabel("Longitude")

plt.show()